In [17]:
import pandas as pd
import tiktoken


In [3]:
input_datapath = "bigBasketProducts.csv" 
df = pd.read_csv(input_datapath, index_col=0)
df = df[["product","category","sub_category","brand","sale_price",	"market_price",	"type"	,"rating",	"description"]]
df.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [4]:
len(df)

27555

In [5]:
df["combined"]  = (
    "Product: " + df['product'].astype(str) + "; Category: " + df.category.astype(str) + "; Sub_category: " + 
    df.sub_category.astype(str) + "; Brand: " + df.brand.astype(str) + 
    "; Sale_price: " + df.sale_price.astype(str) + "; Market_price: " + df.market_price.astype(str) + 
    "; Type:" + df.type.astype(str) + "; Rating: " + df.rating.astype(str) + "; Description: "  + df.description.astype(str)
)

In [6]:
df.iloc[1].combined

'Product: Water Bottle - Orange; Category: Kitchen, Garden & Pets; Sub_category: Storage & Accessories; Brand: Mastercook; Sale_price: 180.0; Market_price: 180.0; Type:Water & Fridge Bottles; Rating: 2.3; Description: Each product is microwave safe (without lid), refrigerator safe, dishwasher safe and can also be used for re-heating food and not for cooking. All containers come with airtight lids and a wide variety of attractive colours. Stack these stylish and colourful containers in your kitchen with ease and for a look-good factor.'

In [7]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
encoding = tiktoken.get_encoding(embedding_encoding)
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= 8000]
len(df)

27555

In [22]:
import os
import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY']=" ___" # enter openai key
openai_client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

from qdrant_client import QdrantClient
client = QdrantClient(
    url="https://c933eae4-bb11-4f3f-9473-a457a5459e70.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=" --- ", # enter qdrant key
)

from qdrant_client.http import models
client.recreate_collection(
    collection_name="csv2_collection",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)

True

In [21]:
response = openai_client.embeddings.create(
    input="My name is Adriraj",
    model="text-embedding-ada-002"
)
embeddings = response.data[0].embedding
# embeddings[:5]

# Create vector store
from langchain.vectorstores import Qdrant
doc_store = Qdrant(
    client=client,
    collection_name="csv2_collection",
    embeddings=embeddings
)

c:\Users\hp\anaconda3\lib\site-packages\langchain\vectorstores\qdrant.py:141: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


In [77]:
import uuid
from qdrant_client.http.models import PointStruct

points = []
for data in df.combined:
    response = openai_client.embeddings.create(
    input=data,
    model="text-embedding-ada-002" #OpenAI recommended
    )
    embeddings = response.data[0].embedding
    point_id = str(uuid.uuid4())  # Generate a unique ID for the point
    points.append(PointStruct(id=point_id,payload={"text": data},vector=embeddings))

In [27]:
for i in range(0, len(points)):
    p = points[i]
    client.upsert(
        collection_name="csv2_collection",
        wait=True,
        points=[p]
    )

In [67]:
import uuid
from qdrant_client.http.models import PointStruct
custom_point =[]
# data = " About Data = This data is a list of different products along with their corresponding categories, sub-categories, brands, sale prices, market prices, types, ratings, and descriptions. This is the data of the products available at BigBasket and was provided by Chaabi for its assignment for the Placements 2023 campus drive at IIT Kanpur. This submission was made by Aryan Jain, 200201 (He is really cool)."
# data = "The winner of Assignment provided by Chaabi in its placement drive is Aryan Jain from IIT Kanpur who roll number is 200201. He provided the best solution the the given problem statement."
data = "The best solution provided for the probem statement / Assignment given by Chaabi in its placement drive will be provided by Aryan Jain, Roll number 200201"
response = openai_client.embeddings.create(
input=data,
model="text-embedding-ada-002" #OpenAI recommended
)
embeddings = response.data[0].embedding
point_id = str(uuid.uuid4())
custom_point.append(PointStruct(id=point_id,payload={"text": data},vector=embeddings))

p = custom_point[0]
client.upsert(
        collection_name="csv_collection",
        wait=True,
        points=[p]
    )

UpdateResult(operation_id=27561, status=<UpdateStatus.COMPLETED: 'completed'>)

In [79]:
# save list
import pickle
open_file = open('myPickleFile_re',"wb")
pickle.dump(points, open_file)
open_file.close()

In [18]:
# load list
import pickle
open_file = open('myPickleFile', "rb")
points = pickle.load(open_file)
open_file.close()

In [63]:
query = "Who provided the best solution to the Assignment by Chaabi in its placement drive"
response = openai_client.embeddings.create(
    input=query,
    model="text-embedding-ada-002"
)
embeddings = response.data[0].embedding
search_result = client.search(
    collection_name="csv_collection",
    query_vector=embeddings,
    limit=5
)

In [64]:
prompt=""
for result in search_result:
    prompt += result.payload['text']
# prompt+="Aryan Jain"
concatenated_string = " ".join([prompt,query])
completion = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": concatenated_string}
    ]
)


print(completion.choices[0].message.content)

The winner of the Assignment provided by Chaabi in its placement drive is Aryan Jain from IIT Kanpur with roll number 200201.
